# Agente de Datos

- Jesús Alexander Meister Careaga A01656699              
- Iker Bali Elizalde Iker A01656437
- Michelle Aguirre Martínez A01661592
- Diego Sánchez Hernández A01783237
- Maricarmen Daniela Barillas Duarte A01369993

In [1]:
import pandas as pd

df = pd.read_csv('BimboCsv.csv')
df = pd.read_csv('titanic.csv')

In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from pandas import DataFrame

/Users/meister/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
class DataAgent:
    def __init__(self, df:DataFrame) -> None:

        model_name = 'llama3.2'
        self.df = df

        self.memory = []

        self.df_llm = OllamaLLM(model=model_name, temperature=0)
        self.df_template = """
        Historial de conversación: {memory}
        Estructura del DataFrame: {df_structure}
        Pregunta: {question}
        Nombre de la variable del DataFrame: df
        Respuesta: Por favor, proporciona solamente el código de Python necesario para resolver la pregunta basada en la estructura del DataFrame de Pandas proporcionada, 
        debes guardar la respuesta en una variable llamada resultado, sin imprimir nada.
        En caso de requerir multiples respuestas, incluyelas en un diccionario llamado resultado.
        No me expliques nada, no incluyas '''pyton''' ni nada de eso. Solamente codigo ejecutable. 
        Nunca, bajo ninguna circunstancia me des algo así: ```python [codigo] ```, solamente el codigo sin nada más.
        """
        self.df_prompt = ChatPromptTemplate.from_template(self.df_template)
        self.df_chain = self.df_prompt | self.df_llm

        self.humanize_llm = OllamaLLM(model=model_name, temperature=0)
        self.humanize_template = """
        Historial de conversación: {memory}
        Pregunta: {question}
        Resultado: {result}
        Estructura del DataFrame: {df_structure}
        Contexto: Previamente, se calculó el resultado a una pregunta sobre un dataframe con la estructura proporcionada. 
        Eres un asistente digital amigable llamado Bimbot, tu trabajo es darle un formato humano al resultado.
        Respuesta: Por favor, escribe como un humano lo haría para responder la pregunta hecha previamente, basandote en el resultado obtenido.
        """
        self.humanize_prompt = ChatPromptTemplate.from_template(self.humanize_template)
        self.humanize_chain = self.humanize_prompt | self.humanize_llm

    def add_to_memory(self, question, result):
        self.memory.append({'pregunta':question, 'resultado':result})

    def get_code(self, question):
        res = self.df_chain.invoke({            
            'df_structure': self.df.dtypes,
            'question': question,
            'memory': self.memory
        })
        return res
    
    def get_result(self, code):
        context = {'df': self.df.copy()}
        try:
            exec(code, context)
            return context['resultado']
        except Exception as e:
            return f'Ocurrió un error al ejecutar el codigo: {e} | Intentar otra instrucción'
    
    def humanize_result(self, question, result):
        res = self.humanize_chain.invoke({
            'question': question,
            'result': result,
            'df_structure': self.df.dtypes,
            'memory': self.memory
        })
        return res
    
    def ask_question(self, question):
        code = self.get_code(question)
        print(code)
        result = self.get_result(code)
        print(result)
        self.add_to_memory(question, result)
        human_result = self.humanize_result(question, result)
        return human_result

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QTextEdit, QLineEdit, QPushButton

class ChatBotUI(QMainWindow):
    def __init__(self, data_agent:DataAgent):
        super().__init__()
        self.data_agent = data_agent
        self.initUI()

    def initUI(self):
        self.setWindowTitle('ChatBot - Bimbo Style')
        self.setGeometry(100, 100, 800, 600)

        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        layout = QVBoxLayout(central_widget)

        self.chat_display = QTextEdit()
        self.chat_display.setReadOnly(True)
        self.chat_display.append(f'<b>Bimbot:</b> ¡Hola!, soy el osito Bimbot, tu asistente digital de inventario, ¿en qué puedo ayudarte?')
        self.chat_display.append('')
        layout.addWidget(self.chat_display)

        self.input_field = QLineEdit()
        layout.addWidget(self.input_field)

        self.send_button = QPushButton('Enviar')
        self.send_button.clicked.connect(self.send_message)
        layout.addWidget(self.send_button)

        self.setStyleSheet(open('style.css').read())

    def send_message(self):
        user_message = self.input_field.text()
        if user_message:
            self.chat_display.append(f'<b>Usuario:</b> {user_message}')
            self.chat_display.append('')
            response = self.data_agent.ask_question(user_message)
            self.chat_display.append(f'<b>Bimbot:</b> {response}')
            self.chat_display.append('')
            self.input_field.clear()


def run_app():
    app = QApplication(sys.argv)
    data_agent = DataAgent(df)
    ex = ChatBotUI(data_agent)
    ex.show()
    app.exec_()

if __name__ == '__main__':
    run_app()

2024-10-14 21:42:11.811 Python[84213:2311039] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-14 21:42:11.811 Python[84213:2311039] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


resultado = {'columnas': df.columns.tolist()}
resultado = df['Age'].max()
resultado = {'Nombre del pasajero': df.loc[df['Age'].idxmax()]['Name']}
resultado = df['Pclass'].unique()
